In [ ]:
# TO DO:
# - [x] save the weights locally to avoid download each time
# - [ ] back-propagation on one exemple
# - [ ] cycle of back-porpagation on a set of exemple
# - [ ] metric logging of progress during training
# - [ ] GPU distribution of training
# - [ ] add attention masks if need be
# - [ ] see if possible to use GPT-3 with Open-AI free account
# - [ ] dans l'ensemble des documents, reprendre les lettres utilisées par Torch pour plus de clarté: N = nombre d'exemples / observations ; C = nombre de classes (nombre de mots de chaque dictionnaire), cf. https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html#torch.nn.functional.cross_entropy


## 1. Chargement des données d'entraînement

Les images sont issues de COCO, et ont été filtrées pour (tenter de) ne retenir que des images représentant des paysages.

In [ ]:
# !wget https://minio.lab.sspcloud.fr/cthiounn2/archive_val.zip
# !unzip -o ../image_data/archive_val.zip -d ../image_data/

--2022-01-29 09:26:15--  https://minio.lab.sspcloud.fr/cthiounn2/archive_val.zip
Resolving minio.lab.sspcloud.fr (minio.lab.sspcloud.fr)... 185.24.184.229, 185.24.184.228
Connecting to minio.lab.sspcloud.fr (minio.lab.sspcloud.fr)|185.24.184.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85830108 (82M) [binary/octet-stream]
Saving to: ‘archive_val.zip’

archive_val.zip     100%[===================>]  81.85M  13.9MB/s    in 6.5s    

2022-01-29 09:26:22 (12.6 MB/s) - ‘archive_val.zip’ saved [85830108/85830108]



In [4]:
verbose = 0
text_token_length = 255


## 1. Entraînement

L'entraînement consiste en:

1. Encoder le texte en tokens-texte
2. Encoder les images en tokens-image
3. Modéliser l'ensemble de façon auto-régressive

### 1.1. Encodage du texte

Nous utilison `BartTokenizer` pour l'encodage du texte comme mini Dall-E. Le Dall-E original utilise selon l'article du "_BPE-encoding_" (byte-pair encoding, c'est à dire strictement parlant des paires de caractères), ce qui peut s'interpréter comme l'utilisation du modèle GPT-3, qui repose lui-aussi sur un encodage proche d'un _BPE encoding_. Malheureusement, GPT-3 n'est pas disponible au grand public.

In [6]:
!git clone https://huggingface.co/facebook/bart-large-cnn ../models/facebook/bart-large-cnn

Clonage dans '../models/facebook/bart-large-cnn'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 31), reused 0 (delta 0) 342.00 Kio/s
Dépaquetage des objets: 100% (75/75), 1.06 Mio | 379.00 Kio/s, fait.
^C
Vous pouvez inspecter ce qui a été extrait avec 'git status'
et réessayer avec 'git restore --source=HEAD :/'



In [7]:
from transformers import BartTokenizer
import torch

# https://huggingface.co/transformers/v2.11.0/model_doc/bart.html

caption = "A Emperor penguin standing on the ice"

# tokenizer = BartTokenizer.from_pretrained('../models/facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

text_tokens = tokenizer(caption, max_length=text_token_length, padding='max_length')['input_ids']
text_tokens = torch.as_tensor(text_tokens) # BPE-encoding
if verbose >= 2:
    print(text_tokens)


### 1.2 Encodage des images

Nous utilisons alternativement 

In [12]:
# Create a local copy
# # ! curl https://cdn.openai.com/dall-e/encoder.pkl --create-dirs -o ../models/openai/dall-e/encoder.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  205M  100  205M    0     0  1107k      0  0:03:09  0:03:09 --:--:-- 1214k      0  0:02:56  0:00:17  0:02:39 1215k:00:22  0:02:34 1203k32 65.8M    0     0  1074k      0  0:03:15  0:01:02  0:02:13  350k 0:03:26  0:01:13  0:02:13 1196k0  1038k      0  0:03:22  0:01:23  0:01:59 1180k3:19  0:01:29  0:01:50 1218k 0  1079k      0  0:03:14  0:01:50  0:01:24 1193k0:02:15  0:00:55 1212k03:10  0:02:16  0:00:54 1208k     0  1099k      0  0:03:11  0:02:55  0:00:16 1184k0  0:03:00  0:00:10 1174k 0:03:09 --:--:-- 1218k


In [13]:
## TRAINING 
import io
import os, sys
import requests
import PIL

import torch
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import torch.nn.functional as F

from dall_e          import map_pixels, unmap_pixels, load_model
from IPython.display import display, display_markdown

dev = torch.device('cpu')


# get token ids for images (= encode)

encoder = load_model("../models/openai/dall-e/encoder.pkl", dev)


target_image_size = 256

# scale images down to 256x256 (cropping the uneven dimension)
# we might get problems with some images from the COCO datasets
# ignore these images as a first approximation
# or reduce the image resolution ?

def preprocess(img):
    s = min(img.size)
    
    if s < target_image_size:
        raise ValueError(f'min dim for image {s} < {target_image_size}')
        
    r = target_image_size / s
    s = (round(r * img.size[1]), round(r * img.size[0]))
    img = TF.resize(img, s, interpolation=PIL.Image.LANCZOS)
    img = TF.center_crop(img, output_size=2 * [target_image_size])
    img = torch.unsqueeze(T.ToTensor()(img), 0)
    return map_pixels(img)

# replace by direct reading from disk
# persist images to disk in the first place
def download_image(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return PIL.Image.open(io.BytesIO(resp.content))

x = preprocess(download_image('https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iKIWgaiJUtss/v2/1000x-1.jpg'))

z_logits = encoder(x)
z = torch.argmax(z_logits, axis=1)
#z = F.one_hot(z, num_classes=encoder.vocab_size).permute(0, 3, 1, 2).float()

# pad text to fixed length with an additional id and bind text
# and image tokens together

# model the sequence with a transformer model



/usr/local/lib/python3.9/site-packages/torchvision/transforms/functional.py:364: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [14]:

image_tokens = z.flatten()
# all_tokens=  torch.cat( (text_tokens,image_tokens) )
# 
# if verbose > 2:
#    print(all_tokens.shape)

### 1.3 Entraînement avec **une** image

In [18]:
# QUESTIONS:
# - Is it a problem that there is potentiel overlap between the indices of image and text-tokens ?

from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained('../models/facebook/bart-large-cnn')

In [22]:
text_tokens = text_tokens.unsqueeze(0)
text_tokens.shape

torch.Size([1, 255])

In [24]:
image_tokens = image_tokens.unsqueeze(0)
image_tokens.shape

torch.Size([1, 1024])

In [25]:
predict = model(input_ids = text_tokens, decoder_input_ids = image_tokens)

In [27]:
predict.logits.shape # les prédictions sont dans le plongement des tokens-texte, pas des tokens-image !

torch.Size([1, 1024, 50264])

In [28]:
tokenizer.vocab_size # 50265

50265

In [29]:
encoder.vocab_size # 8192

8192

Le modèle BART utilisé est prévu pour utiliser un seul et même dictionnaire pour faire des résumés de texte. Or ici nous voulons utiliser un dictionnaire nouveau pour le décodage... Il nous faut donc remplacer la dernière couche du modèle, pour prédire non pas la proba du prochain token parmi 50264/5 (?) tokens du vocabulaire de token-texte, mais plutôt la proba du prochain token parmi 8192 tokens-image.

In [30]:
model.lm_head

Linear(in_features=1024, out_features=50264, bias=False)

In [33]:
import torch.nn as nn
model.lm_head = nn.Linear(in_features=1024, out_features=16384, bias=False)
# we just change the prediction size of the last layer
model.final_logits_bias = torch.rand(16384)
# for some reason, the bias are stored outside the neural network layer

In [34]:
predict = model(input_ids = text_tokens, decoder_input_ids = image_tokens)
predict.logits.shape

torch.Size([1, 1024, 16384])

Calculons la précision de ce modèle (jusqu'à présent non-entraîné pour cette tâche spécifique) :

In [40]:
from torch.nn.functional import cross_entropy
loss = cross_entropy(predict.logits.squeeze(), image_tokens.squeeze())

In [41]:
loss

tensor(9.8533, grad_fn=<NllLossBackward>)

In [42]:
loss.backward() # > 30 sec

In [47]:
# je ne comprends pas pourquoi la fonction de perte ne change pas
# après appel à la méthode .backward()
model.forward()
predict = model(input_ids = text_tokens, decoder_input_ids = image_tokens)
# par ailleurs l'appel à squeeze() ici sous-entend qu'on ne va pas pouvoir 
# calculer la fonction de perte en une seule fois sur un batch
# ou alors qu'il va falloir ruser...
loss = cross_entropy(predict.logits.squeeze(), image_tokens.squeeze())
loss

AttributeError: 'NoneType' object has no attribute 'new_zeros'

tensor(9.8533, grad_fn=<NllLossBackward>)

In [ ]:
# Pus tard: gestion de l'attention
# torch.ones_like(all_tokens)

In [ ]:
## INFERENCE

# get token ids for texts (= encode)

# generate the next terms in the sequence with a random seed

# get image from token ids (= decode)